In [2]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import jax
import jax_flock as jf
import functools
import stats

params = jf.Params(
    weight_forward  = 0.20,
    weight_separate = 1.00,
    weight_align    = 0.30,
    weight_cohere   = 0.60,
    weight_avoid    = 0.80
)

config = jf.Config(
    boid_count = 100,
    sphere_radius = 30.0
)

time_step = 1 / 60

key = jax.random.PRNGKey(0)
state = jf.init_state(config, key)
def next_state(state):
    return jf.next_state(params, config, time_step, state)

time_average = jax.jit(stats.time_average, static_argnames=['next_fn', 'n_steps', 'J_fn'])
a = time_average(state, next_state, 1000, jf.avg_sep).block_until_ready()

In [6]:
print(jax.devices())
%timeit time_average(state, next_state, 1000, jf.avg_sep).block_until_ready()

[cuda(id=0)]
78.3 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
